In [1]:
from simple.binance import getHistMonth
from simple.plotly import chartFigure, updateLines
import pandas as pd
from simple.pretty import pmap
from simple.funcs import hurst

from statsmodels.tsa.stattools import adfuller
from ipyslickgrid import show_grid
from ipywidgets import VBox

In [2]:
frame = '1h'

tickers = [
    '1000BTTCUSDT', '1000SHIBUSDT', '1000XECUSDT', '1INCHUSDT', 'AAVEUSDT', 'ADABUSD', 'ADAUSDT', 'AKROUSDT',
    'ALGOUSDT', 'ALICEUSDT', 'ALPHAUSDT', 'ANKRUSDT', 'ANTUSDT', 'API3USDT', 'ARPAUSDT', 'ARUSDT', 'ATAUSDT', 'ATOMUSDT',
    'AUDIOUSDT', 'AVAXUSDT', 'AXSUSDT', 'BAKEUSDT', 'BALUSDT', 'BANDUSDT', 'BATUSDT', 'BCHUSDT', 'BELUSDT', 'BLZUSDT',
    'BNBBUSD', 'BNBUSDT', 'BTCBUSD', 'BTCDOMUSDT', 'BTCSTUSDT', 'BTCUSDT', 'BTSUSDT', 'C98USDT', 'CELOUSDT', 'CELRUSDT',
    'CHRUSDT', 'CHZUSDT', 'COMPUSDT', 'COTIUSDT', 'CRVUSDT', 'CTKUSDT', 'CTSIUSDT', 'CVCUSDT', 'DASHUSDT', 'DEFIUSDT',
    'DENTUSDT', 'DGBUSDT', 'DODOUSDT', 'DOGEBUSD', 'DOGEUSDT', 'DOTUSDT', 'DUSKUSDT', 'DYDXUSDT', 'EGLDUSDT', 'ENJUSDT',
    'ENSUSDT', 'EOSUSDT', 'ETCUSDT', 'ETHBUSD', 'ETHUSDT', 'FILUSDT', 'FLMUSDT', 'FLOWUSDT', 'FTMUSDT', 'FTTBUSD',
    'GALAUSDT', 'GRTUSDT', 'GTCUSDT', 'HBARUSDT', 'HNTUSDT', 'HOTUSDT', 'ICPUSDT', 'ICXUSDT', 'IMXUSDT', 'IOSTUSDT',
    'IOTAUSDT', 'IOTXUSDT', 'KAVAUSDT', 'KLAYUSDT', 'KNCUSDT', 'KSMUSDT', 'LINAUSDT', 'LINKUSDT', 'LITUSDT', 'LPTUSDT',
    'LRCUSDT', 'LTCUSDT', 'LUNAUSDT', 'MANAUSDT', 'MASKUSDT', 'MATICUSDT', 'MKRUSDT', 'MTLUSDT', 'NEARUSDT', 'NEOUSDT',
    'NKNUSDT', 'OCEANUSDT', 'OGNUSDT', 'OMGUSDT', 'ONEUSDT', 'ONTUSDT', 'PEOPLEUSDT', 'QTUMUSDT', 'RAYUSDT', 'REEFUSDT',
    'RENUSDT', 'RLCUSDT', 'ROSEUSDT', 'RSRUSDT', 'RUNEUSDT', 'RVNUSDT', 'SANDUSDT', 'SCUSDT', 'SFPUSDT', 'SKLUSDT',
    'SNXUSDT', 'SOLBUSD', 'SOLUSDT', 'SRMUSDT', 'STMXUSDT', 'STORJUSDT', 'SUSHIUSDT', 'SXPUSDT', 'THETAUSDT', 'TLMUSDT',
    'TOMOUSDT', 'TRBUSDT', 'TRXUSDT', 'UNFIUSDT', 'UNIUSDT', 'VETUSDT', 'WAVESUSDT', 'XEMUSDT', 'XLMUSDT', 'XMRUSDT',
    'XRPBUSD', 'XRPUSDT', 'XTZUSDT', 'YFIIUSDT', 'YFIUSDT', 'ZECUSDT', 'ZENUSDT', 'ZILUSDT', 'ZRXUSDT'
]

In [3]:
def getClose(ticker: str) -> pd.Series:
    return getHistMonth('2022-02', '2022-03', ticker, frame, close_only=True, spot=False)

In [4]:
# parallel load data from binance archive
Close = pd.concat(pmap(getClose, tickers), axis=1)
Close.head()

  0%|          | 0/147 [00:00<?, ?it/s]

,1000BTTCUSDT,1000SHIBUSDT,1000XECUSDT,1INCHUSDT,AAVEUSDT,ADABUSD,ADAUSDT,AKROUSDT,ALGOUSDT,ALICEUSDT,...,XMRUSDT,XRPBUSD,XRPUSDT,XTZUSDT,YFIIUSDT,YFIUSDT,ZECUSDT,ZENUSDT,ZILUSDT,ZRXUSDT
DT,,,,,,,,,,,,,,,,,,,,,
2022-02-01 00:00:00,0.002135,0.021436,0.07660,1.6872,158.63,1.0446,1.0441,0.01329,0.9474,6.979,...,146.28,0.6143,0.6143,3.514,2326.3,24777.0,94.75,41.059,0.04479,0.5666
2022-02-01 01:00:00,0.002131,0.021288,0.07640,1.6874,159.70,1.0431,1.0432,0.01329,0.9532,7.074,...,145.52,0.6163,0.6161,3.500,2336.3,24808.0,94.85,40.990,0.04483,0.5729
2022-02-01 02:00:00,0.002134,0.021445,0.07685,1.6938,161.09,1.0451,1.0446,0.01342,0.9514,7.105,...,147.51,0.6208,0.6205,3.514,2341.0,24852.0,95.35,41.135,0.04527,0.5771
2022-02-01 03:00:00,0.002201,0.021733,0.07674,1.7056,163.74,1.0563,1.0556,0.01364,0.9655,7.208,...,147.72,0.6262,0.6260,3.542,2344.4,25031.0,96.30,41.348,0.04548,0.5848
2022-02-01 04:00:00,0.002175,0.021622,0.07660,1.6943,162.42,1.0536,1.0532,0.01377,0.9720,7.245,...,144.94,0.6239,0.6236,3.535,2323.9,24955.0,95.85,41.038,0.04531,0.5838


In [5]:
# parallel calculate Hurst Exponent value
Hurst = pd.DataFrame(pmap(lambda t: hurst(Close[t].values), Close.columns), index=Close.columns, columns=['Hurst Exponent'])
Hurst

  0%|          | 0/147 [00:00<?, ?it/s]

LinAlgError: SVD did not converge in Linear Least Squares

In [ ]:
# parallel run of Augmented Dickey-Fuller test
ADF = pd.DataFrame(pmap(adfuller, Close.dropna().values.T),
                   index=Close.columns,
                   columns=['ADF', 'P-value', 'Used Lag', 'Observations', 'Critical Values', 'Best IC'])
ADF

In [ ]:
fig = chartFigure(Close=dict(color='lightblue', width=3), height=400)

In [ ]:
def on_changed(event, grid):
    """Redraw chart with selected ticker's data"""

    changed = grid.get_changed_df()
    k = event['new'][0]
    selected = changed.iloc[k:k + 1].to_dict('records')[0]
    ticker = selected['index']
    updateLines(fig, Close=Close[ticker])
    fig.layout.title.text = ticker

In [ ]:
X = Hurst.join(ADF).reset_index()
grid = show_grid(X, grid_options={'editable': False, 'forceFitColumns': True, 'multiSelect': False, 'rowHeight': 24, 'maxVisibleRows': 14},
                 column_options={'defaultSortAsc': False},
                 column_definitions={'Critical Values': {'width': 500}})

grid.on('selection_changed', on_changed)
grid.change_selection([0])

In [ ]:
VBox([fig, grid])